In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from modules.my_spark_regression import *
from modules.my_pyspark import *
from modules.my_drawer import MyDrawer

In [3]:
spark = MyPySpark(session=True, sql=True)
drawer = MyDrawer()

In [4]:
file_path = r'./data/Ecommerce_Customers.csv'


In [5]:
data = spark.readFile(file_path)


In [6]:
input_features = [
    'Avg Session Length',
    'Time on App',
    'Time on Website',
    'Length of Membership'
]

target_feature = 'Yearly Amount Spent'

In [7]:
lr_model = MySparkRegression(data, input_features, target_feature)

In [8]:
lr_model.prepareData()

In [9]:
lr_model.trainTestSplit()

In [10]:
lr_model.train_data.select(lr_model.target_feature).describe().withColumnRenamed(lr_model.target_feature, lr_model.target_feature + " (new)").show()

+-------+-------------------------+
|summary|Yearly Amount Spent (new)|
+-------+-------------------------+
|  count|                      364|
|   mean|       500.55725933512986|
| stddev|        79.47187244331258|
|    min|       256.67058229005585|
|    max|        765.5184619388373|
+-------+-------------------------+



In [22]:
lr_model.describeTarget()

+-------+---------------------------+--------------------------+
|summary|Yearly Amount Spent (train)|Yearly Amount Spent (test)|
+-------+---------------------------+--------------------------+
|  count|                        364|                       136|
|   mean|         500.55725933512986|        495.98659361256034|
| stddev|          79.47187244331258|         79.08881448784598|
|    min|         256.67058229005585|         282.4712457199145|
|    max|          765.5184619388373|         725.5848140556806|
+-------+---------------------------+--------------------------+



In [12]:
sparkk = spark.session

In [13]:
 from pyspark.sql.types import *

In [14]:
schema=StructType().add("user_id","string").add("country","string").add("browser", "string").add("OS",'string').add("age", "integer")

In [15]:
df=sparkk.createDataFrame([("A203",'India',"Chrome","WIN",33),("A201",'China',"Safari","MacOS",35),("A205",'UK',"Mozilla","Linux",25)],schema=schema)

In [16]:
df.show()

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A203|  India| Chrome|  WIN| 33|
|   A201|  China| Safari|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [17]:
df2=sparkk.createDataFrame([("A203",'India',"Chrome","WIN",33),("A201",'China',"Safari","MacOS",35),("A205",'UK',"Mozilla","Linux",25)],schema=schema)

In [18]:
new_df = df.join(df2, on='OS')

In [19]:
new_df.show()

+-----+-------+-------+-------+---+-------+-------+-------+---+
|   OS|user_id|country|browser|age|user_id|country|browser|age|
+-----+-------+-------+-------+---+-------+-------+-------+---+
|Linux|   A205|     UK|Mozilla| 25|   A205|     UK|Mozilla| 25|
|  WIN|   A203|  India| Chrome| 33|   A203|  India| Chrome| 33|
|MacOS|   A201|  China| Safari| 35|   A201|  China| Safari| 35|
+-----+-------+-------+-------+---+-------+-------+-------+---+



In [20]:
new_dff = new_df.describe().withColumnRenamed('OS', 'os')

In [21]:
new_dff.show()

+-------+-----+-------+-------+-------+-----------------+-------+-------+-------+-----------------+
|summary|   os|user_id|country|browser|              age|user_id|country|browser|              age|
+-------+-----+-------+-------+-------+-----------------+-------+-------+-------+-----------------+
|  count|    3|      3|      3|      3|                3|      3|      3|      3|                3|
|   mean| null|   null|   null|   null|             31.0|   null|   null|   null|             31.0|
| stddev| null|   null|   null|   null|5.291502622129181|   null|   null|   null|5.291502622129181|
|    min|Linux|   A201|  China| Chrome|               25|   A201|  China| Chrome|               25|
|    max|  WIN|   A205|     UK| Safari|               35|   A205|     UK| Safari|               35|
+-------+-----+-------+-------+-------+-----------------+-------+-------+-------+-----------------+

